


HerokuによるBasic認証と定期実行を実装

- https://sitest.jp/blog/?p=9238




## ■ Webスクレイピング


In [1]:
!pip3 install beautifulsoup4

     |████████████████████████████████| 115 kB 5.0 MB/s eta 0:00:01


In [2]:
import requests
from bs4 import BeautifulSoup

In [4]:
url = 'https://scraping-for-beginner.herokuapp.com/udemy'

In [9]:
# スクレイピング対象の URL にリクエストを送り HTML を取得する
res = requests.get(url)

In [10]:
# レスポンスの HTML から BeautifulSoup オブジェクトを作る
soup = BeautifulSoup(res.text, 'html.parser')

## ■ データ取得（Udemy情報）

In [11]:
soup.find_all('p', {'class': 'subscribers'})

[<p class="subscribers">受講生の数：11306</p>]

In [13]:
n_subscriber = soup.find('p', {'class': 'subscribers'}).text
n_subscriber

'受講生の数：11306'

In [16]:
n_subscriber = int(n_subscriber.split('：')[1])
n_subscriber

11306

In [17]:
n_review = soup.find('p', {'class': 'reviews'}).text
n_review

'レビューの数：1955'

In [18]:
n_review = int(n_review.split('：')[1])
n_review

1955

## ■ 1つのデータ取得（ECサイト情報）

In [81]:
url_ec = 'http://scraping.official.ec/'

In [82]:
res = requests.get(url_ec)

In [83]:
soup = BeautifulSoup(res.text, 'html.parser')

In [84]:
itemList = soup.find('ul', {'id': 'itemList'})

In [85]:
len(itemList.find_all('li'))

4

In [86]:
items = itemList.find_all('li')

In [111]:
item = items[0]

#### まずは商品タイトル

In [88]:
item.find_all('p', {'class': 'items-grid_itemTitleText_b58666da'})

[<p class="items-grid_itemTitleText_b58666da">プログラミング学習の相談</p>]

In [89]:
title = item.find('p', {'class': 'items-grid_itemTitleText_b58666da'}).text
title

'プログラミング学習の相談'

#### 次に価格

In [90]:
item.find_all('p', {'class': 'items-grid_price_b58666da'})

[<p class="items-grid_price_b58666da">¥ 30,000</p>]

In [91]:
price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
price

'¥ 30,000'

In [92]:
price = int(price.replace('¥ ', '').replace(',', ''))
price

30000

### 詳細ページリンク

In [141]:
len(item.find_all('a'))

1

In [143]:
link = item.find('a')['href']
link

'https://scraping.official.ec/items/40792454'

### 最後に在庫

In [117]:
items[0].find('p', {'class': 'items-grid_soldOut_b58666da'}) == None

True

In [118]:
items[3].find('p', {'class': 'items-grid_soldOut_b58666da'}) == None

False

In [119]:
is_stock = item.find('p', {'class': 'items-grid_soldOut_b58666da'}) == None
is_stock

False

In [120]:
is_stock = '在庫あり' if is_stock == True else '在庫なし'
is_stock

'在庫なし'

## ■ 複数のデータ取得（ECサイト情報）

In [121]:
url_ec = 'http://scraping.official.ec/'
res = requests.get(url_ec)
soup = BeautifulSoup(res.text, 'html.parser')
itemList = soup.find('ul', {'id': 'itemList'})
items = itemList.find_all('li')

title = item.find('p', {'class': 'items-grid_itemTitleText_b58666da'}).text
price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
price = int(price.replace('¥ ', '').replace(',', ''))
link = item.find('a')['href']

In [144]:
url_ec = 'http://scraping.official.ec/'
res = requests.get(url_ec)
soup = BeautifulSoup(res.text, 'html.parser')
itemList = soup.find('ul', {'id': 'itemList'})

data_ec = []
items = itemList.find_all('li')
for item in items:
    datum_ec = {}
    datum_ec['title'] = item.find('p', {'class': 'items-grid_itemTitleText_b58666da'}).text
    price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
    datum_ec['price'] = int(price.replace('¥ ', '').replace(',', ''))
    datum_ec['link'] = item.find('a')['href']
    is_stock = item.find('p', {'class': 'items-grid_soldOut_b58666da'}) == None
    datum_ec['is_stock'] = '在庫あり' if is_stock == True else '在庫なし'
    data_ec.append(datum_ec)

In [145]:
data_ec

[{'title': 'プログラミング学習の相談',
  'price': 30000,
  'link': 'https://scraping.official.ec/items/40792108',
  'is_stock': '在庫あり'},
 {'title': '顧問 & アドバイザリー',
  'price': 100000,
  'link': 'https://scraping.official.ec/items/40792153',
  'is_stock': '在庫あり'},
 {'title': 'Udemy講座まとめ買いパック',
  'price': 8000,
  'link': 'https://scraping.official.ec/items/40792284',
  'is_stock': '在庫あり'},
 {'title': 'YouTubeコンサルティング',
  'price': 15000,
  'link': 'https://scraping.official.ec/items/40792454',
  'is_stock': '在庫なし'}]

## ■ 一連の流れを関数化
### Udemy

In [126]:
def get_data_udemy():
    data_udemy = {}
    url = 'https://scraping-for-beginner.herokuapp.com/udemy'
    res = requests.get(url)

    soup = BeautifulSoup(res.text, 'html.parser')
    soup.find_all('p', {'class': 'subscribers'})

    n_subscriber = soup.find('p', {'class': 'subscribers'}).text
    n_subscriber = int(n_subscriber.split('：')[1])

    n_review = soup.find('p', {'class': 'reviews'}).text
    n_review = int(n_review.split('：')[1])
    return {
        'n_subscriber': n_subscriber,
        'n_review': n_review
    }

In [127]:
get_data_udemy()

{'n_subscriber': 11323, 'n_review': 1956}

### ECサイト

In [146]:
def get_data_ec():
    url_ec = 'http://scraping.official.ec/'
    res = requests.get(url_ec)
    soup = BeautifulSoup(res.text, 'html.parser')
    itemList = soup.find('ul', {'id': 'itemList'})

    data_ec = []
    items = itemList.find_all('li')
    for item in items:
        datum_ec = {}
        datum_ec['title'] = item.find('p', {'class': 'items-grid_itemTitleText_b58666da'}).text
        price = item.find('p', {'class': 'items-grid_price_b58666da'}).text
        datum_ec['price'] = int(price.replace('¥ ', '').replace(',', ''))
        datum_ec['link'] = item.find('a')['href']
        is_stock = item.find('p', {'class': 'items-grid_soldOut_b58666da'}) == None
        datum_ec['is_stock'] = '在庫あり' if is_stock == True else '在庫なし'
        data_ec.append(datum_ec)
    
    df_ec = pd.DataFrame(data_ec)
    return df_ec

In [147]:
get_data_ec()

,title,price,link,is_stock
0,プログラミング学習の相談,30000,https://scraping.official.ec/items/40792108,在庫あり
1,顧問 & アドバイザリー,100000,https://scraping.official.ec/items/40792153,在庫あり
2,Udemy講座まとめ買いパック,8000,https://scraping.official.ec/items/40792284,在庫あり
3,YouTubeコンサルティング,15000,https://scraping.official.ec/items/40792454,在庫なし


## ■ APIの有効化

- プロジェクトを新規作成
- Google Drive APIを有効化
- Google Sheets APIを有効化
- 認証情報を設定


以下を参考にAPIの有効化等を進める  
https://tanuhack.com/python/operate-spreadsheet/#Google_Sheets_API

参考サイト  
https://tanuhack.com/python/library-gspread/

## ■ シートを共有

https://docs.google.com/spreadsheets/d/1WR2V-m104msfrF--9U7S71VY8KeJ3lVZVhtsdFxs6EY/edit?usp=sharing


## ■ 認証

- gspread：Python3.X系でもスプレッドシートを操作できるライブラリ
- oauth2client：Googleの各種APIへアクセスできるようにするライブラリ


In [ ]:
!pip install gspread
!pip install oauth2client

In [240]:
# import gspread
# from oauth2client.service_account import ServiceAccountCredentials
# import json

In [250]:
from google.oauth2.service_account import Credentials

scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

credentials = Credentials.from_service_account_file(
    'service_account.json',
    scopes=scopes
)

gc = gspread.authorize(credentials)

## ■ スプレッドシートからデータ取得

In [251]:
SP_SHEET_KEY = '1MlEQ5RfMJanndDRZVdpxRR7QMxThs0nbbvcFUGhovpk'
#スプレッドシートの選択(ID)
sh = gc.open_by_key(SP_SHEET_KEY)

In [252]:
SP_SHEET = 'db'
worksheet = sh.worksheet(SP_SHEET)

In [253]:
# シート内のデータ全取得
data= worksheet.get_all_values()

In [255]:
data[:3]

[['date', 'n_subscriber', 'n_review'],
 ['2021/03/09', '11032', '1882'],
 ['2021/03/10', '11049', '1885']]

In [258]:
df = pd.DataFrame(data[1:], columns=data[0])
df[:3]

,date,n_subscriber,n_review
0,2021/03/09,11032,1882
1,2021/03/10,11049,1885
2,2021/03/11,11068,1889


## ■ データの更新

In [259]:
data_udemy = get_data_udemy()
data_udemy

{'n_subscriber': 11343, 'n_review': 1958}

In [260]:
import datetime

In [261]:
today = datetime.date.today()
today

datetime.date(2021, 4, 8)

In [262]:
data_udemy['date'] = datetime.date.today().strftime('%Y/%m/%d')
data_udemy

{'n_subscriber': 11343, 'n_review': 1958, 'date': '2021/04/08'}

In [263]:
df = df.append(data_udemy, ignore_index=True)

In [265]:
df[:3]

,date,n_subscriber,n_review
0,2021/03/09,11032,1882
1,2021/03/10,11049,1885
2,2021/03/11,11068,1889


gspreadだけだとサクッとDataFrameをスプレッドシートに反映できない、、、  
そのため、gspread-dataframeをインストールする。

In [267]:
!pip3 install gspread-dataframe

In [268]:
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [270]:
# 値を書き込む左上のセル番号
first_row = 1
first_col = 1

In [272]:
set_with_dataframe(worksheet, df, row=first_row, col=first_col)

## ■ グラフ化の準備

In [210]:
df_udemy = pd.read_csv('data.csv')
df_udemy[:3]

,date,n_subscriber,n_review
0,2021/03/09,11032,1882
1,2021/03/10,11049,1885
2,2021/03/11,11068,1889


2軸のグラフの書き方を調べた  
`python altair 2 axis`

In [238]:
ymin1 = df_udemy['n_subscriber'].min() - 10
ymax1 = df_udemy['n_subscriber'].max() + 10

ymin2 = df_udemy['n_review'].min() - 10
ymax2 = df_udemy['n_review'].max() + 10

In [239]:
base = alt.Chart(df_udemy).encode(
    alt.X('date:T', axis=alt.Axis(title=None))
)

line1 = base.mark_line(opacity=0.3, color='#57A44C').encode(
    alt.Y('n_subscriber',
          axis=alt.Axis(title='登録者数', titleColor='#57A44C'),
          scale=alt.Scale(domain=[ymin1, ymax1]))
)

line2 = base.mark_line(stroke='#5276A7', interpolate='monotone').encode(
    alt.Y('n_review',
          axis=alt.Axis(title='レビュー数', titleColor='#5276A7'),
          scale=alt.Scale(domain=[ymin2, ymax2]))
)

chart = alt.layer(line1, line2).resolve_scale(
    y = 'independent'
)

chart

alt.LayerChart(...)

## ■ スクリプトファイルにまとめる

- 毎日定期更新し、データを更新するための処理を `scraping.py` にまとめる。
- アプリケーションに必要な部分を `app.py` にまとめる


## ■ GitHub Actionsで定期更新

- アカウントの準備（アカウント作成）
- git init
- git remote add origin https://github.com/imanyun/sample-chatbot.git
- git add .
- git commit -m '1st commit'
- git push origin master


- main.ymlを作る

```yml
on:
  push:
```
を先に


```yml
# ワークフロー名
name: test_line_notify

# 発火タイミング
on:
  schedule:
    - cron: '0 22 * * *'

jobs:
  build:
    # Ubuntuの最新版環境内で処理
    runs-on: ubuntu-latest

    # 実行する処理＆コマンド指定
    steps:
      - uses: actions/checkout@v2
      - name: Set up Python 3.8
        uses: actions/setup-python@v1
        with:
          python-version: 3.8
      - name: Install dependencies
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt
      - name: Run script
        run: |
          # main.pyの実行
          python scraping.py

```

## ■ Herokuにデプロイ


https://towardsdatascience.com/quickly-build-and-deploy-an-application-with-streamlit-988ca08c7e83


- `git init`
- `git remote add heroku REPO_URL`
- `git add .`
- `git commit -m'1st commit'`
- `git push heroku master`



## ■ Heroku Schedulerによる定期実行


- `heroku run bash`

heroku環境でコマンドが打てるようになる。  
いちいち「heroku run」を打つのがカッタルイ人はbashが立ち上がってから、直にコマンドを打とう。

- `ls`

## Basic認証

https://sitest.jp/blog/?p=9238

- `heroku config:set USER=imanyu`
- `heroku config:set PASS=udemy`


